In [3]:
from fastai.vision.all import *

In [4]:
#Imports

import pandas as pd
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random
import torch
from torchvision.models import inception_v3


In [5]:
import torch
print(f"Using CUDA: {torch.cuda.is_available()}")


Using CUDA: True


In [3]:
# Reading in the training data
csv_path = os.path.join('COMP90086_2024_Project_train', 'train.csv')
train_data = pd.read_csv(csv_path)
train_dir = os.path.join('COMP90086_2024_Project_train', 'train')
print(train_data.head())


    id  shapeset  type  total_height  instability_type  cam_angle  \
0   54         2     1             3                 1          1   
1  173         1     1             4                 1          2   
2  245         1     1             4                 1          2   
3  465         2     1             5                 0          1   
4  611         2     1             3                 1          1   

   stable_height  
0              2  
1              1  
2              1  
3              5  
4              1  


In [4]:
# RESNET 50 TRAINING
csv_path = os.path.join('COMP90086_2024_Project_train', 'train.csv')

# Read the CSV file into a DataFrame
train_data = pd.read_csv(csv_path)

# Adding a column with the complete image path
train_data['image_path'] = train_data['id'].apply(lambda x: os.path.join('COMP90086_2024_Project_train', 'train', f"{x}.jpg"))

# Check a few rows to confirm the structure
print(train_data.head())

# Convert your DataFrame to a FastAI DataBlock
dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock),  # Image as input and regression target
    get_x=ColReader('image_path'),        # Path to the image
    get_y=ColReader('stable_height'),     # Target column
    splitter=RandomSplitter(valid_pct=0.2)  # Splitting the data into training and validation sets
)

# Create a DataLoader
dls = dblock.dataloaders(train_data, bs=8, num_workers=0)  # bs=32 means batch size 32, adjust as needed based on your GPU memory

learn = vision_learner(dls, resnet50, metrics=rmse)  # Replace resnet50 with your desired architecture
learn.fine_tune(5)  # Train for a few epochs; adjust as needed

learn.save('resnet50_trained_model')


In [6]:
from fastai.vision.all import *
import pandas as pd
import os
import torch
from torchvision.models import inception_v3

# Define the path to your CSV file
csv_path = os.path.join('COMP90086_2024_Project_train', 'train.csv')

# Read the CSV file into a DataFrame
train_data = pd.read_csv(csv_path)

# Adding a column with the complete image path
train_data['image_path'] = train_data['id'].apply(lambda x: os.path.join('COMP90086_2024_Project_train', 'train', f"{x}.jpg"))

# Define the DataBlock with an appropriate image size for Inception
dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock),   # Image as input and regression target
    get_x=ColReader('image_path'),          # Path to the image
    get_y=ColReader('stable_height'),       # Target column
    splitter=RandomSplitter(valid_pct=0.2), # Splitting the data into training and validation sets
    item_tfms=Resize(299)  # Resize images to 299x299, the required input size for InceptionV3
)

# Create a DataLoader
dls = dblock.dataloaders(train_data, bs=8)  # Adjust the batch size based on your GPU memory

# Load the InceptionV3 model from PyTorch
class InceptionV3Custom(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = inception_v3(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, 1)  # Adjust the final layer for regression

    def forward(self, x):
        # Get the main output and auxiliary output from InceptionV3
        logits, aux_logits = self.model(x)
        return logits  # Only return the main output

# Initialize the modified InceptionV3 model
inception_model = InceptionV3Custom()

# Create a FastAI learner using the modified Inception model
learn = Learner(dls, inception_model, loss_func=MSELossFlat(), metrics=rmse)

# Train the model using fine-tuning
learn.fine_tune(5)  # Adjust the number of epochs as needed

learn.save('inception_v3_trained_model')

c:\Users\julev\anaconda3\envs\cudaEnv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\julev\anaconda3\envs\cudaEnv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\julev/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:08<00:00, 12.1MB/s] 


cuda:0


epoch,train_loss,valid_loss,_rmse,time


KeyboardInterrupt: 